In [1]:
import time
import random
import requests
import csv
import os

import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from parsel import Selector

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys

In [15]:
opts = Options()

# uma sessão do navegador é criada quando instanciamos a classe 'Driver'. Em alguns casos seria necessario incluir
# o caminho para o driver do navegador como parâmetro, mas como incluímos esse arquivo dentro em um dos diretórios
# da variável $PATH (/usr/local/bin) isso não será necessário.
driver = webdriver.Firefox(options=opts)

# acessa a pagina dada pela URL
driver.get('https://www.vagas.com.br')

driver.implicitly_wait(5)

# o método '.find_element()' localiza os elementos da página atual da sessão, baseado no tipo de elemento/ID que o
# objeto possui e pelo nome da classe dado ao elemento do HTML. Após selecionarmos esse objeto, podemos passar
# uma string usando o método '.send_keys()' que simula as teclas pressionadas no teclado.
search_bar = driver.find_element(by=By.ID, value='nova-home-search')
search_bar.send_keys('Analista de dados' + Keys.ENTER)


driver.implicitly_wait(10)

prev_height = -1

# laço para rolar até o final da página
while True:
    
    time.sleep(4)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # rola até o final da página - (x-coord, y-coord)
    new_height = driver.execute_script("return document.body.scrollHeight")  # retona a posição atual em pixels
    
    if new_height == prev_height:
        break

    prev_height = new_height

    try:
        # nessa pagina o "botão" 'maisVagas' não foi configurado usando o elemento <button> do html mas sim um anchor
        # element <a>, o qual possui uma ação configurada via javascript. Para executar a ação atribuída a esse elemento
        # usamos o método '.execute_script()' e passamos como argumentos uma string contendo com script para executar a ação
        # e um argumento que contém os dados da tag html.
        see_more = driver.find_element(By.ID, 'maisVagas')
        driver.execute_script('arguments[0].click()', see_more) # executa a ação que simula o clique de um botão
        
        time.sleep(3)

    except:
        time.sleep(1)


In [10]:
print(see_more)
print(see_more.get_dom_attribute('onclick'))
print(see_more.id)

<selenium.webdriver.remote.webelement.WebElement (session="8a943cbc-c5ef-408e-9dea-962585500743", element="1f21f305-9af5-452f-8ea7-56439498156e")>
ga('send', 'event', 'Pesquisa', 'anuncios');
1f21f305-9af5-452f-8ea7-56439498156e


In [11]:
print(see_more.tag_name)
print(see_more.aria_role)
print(see_more.accessible_name)
print(see_more.location_once_scrolled_into_view)

a
link
MOSTRAR MAIS VAGAS
{'x': 332, 'y': 82}


In [16]:
# html da página
driver.page_source

'<html lang="pt-br" class="vg-browser-firefox vg-window-lg vg-screen-lg"><head><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-VVGK721SF4&amp;cx=c&amp;_slc=1"></script><script type="text/javascript" async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=AW-978727682&amp;l=dataLayer&amp;cx=c"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-6JRW0393LB&amp;cx=c&amp;_slc=1"></script><script src="https://js.hsadspixel.net/fb.js" type="text/javascript" id="hs-ads-pixel-6951874" data-ads-portal-id="6951874" data-ads-env="prod" data-loader="hs-scriptloader" data-hsjs-portal="6951874" data-hsjs-env="prod" data-hsjs-hublet="na1"></script><script src="https://js.hubspot.com/web-interactives-embed.js" type="text/javascript" id="hubspot-web-interactives-loader" crossorigin="anonymous" data-load

In [3]:
import pickle

def save_source_pickle(name, source):
    with open(name, 'wb') as handle:
        pickle.dump(source, handle)

def load_source_pickle(name):
    with open(name, 'rb') as handle:
        return pickle.load(handle)

In [18]:
page_source = driver.page_source
driver.quit()

save_source_pickle('../data/data_raw//page_source.pkl', page_source)

In [4]:
page_source = load_source_pickle('../data/data_raw/page_source.pkl')
page_source

'<html lang="pt-br" class="vg-browser-firefox vg-window-lg vg-screen-lg"><head><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-VVGK721SF4&amp;cx=c&amp;_slc=1"></script><script type="text/javascript" async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=AW-978727682&amp;l=dataLayer&amp;cx=c"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-6JRW0393LB&amp;cx=c&amp;_slc=1"></script><script src="https://js.hsadspixel.net/fb.js" type="text/javascript" id="hs-ads-pixel-6951874" data-ads-portal-id="6951874" data-ads-env="prod" data-loader="hs-scriptloader" data-hsjs-portal="6951874" data-hsjs-env="prod" data-hsjs-hublet="na1"></script><script src="https://js.hubspot.com/web-interactives-embed.js" type="text/javascript" id="hubspot-web-interactives-loader" crossorigin="anonymous" data-load

In [5]:
soup = BeautifulSoup(page_source, 'html.parser')

In [6]:
print(soup.prettify())

<html class="vg-browser-firefox vg-window-lg vg-screen-lg" lang="pt-br">
 <head>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-VVGK721SF4&amp;cx=c&amp;_slc=1" type="text/javascript">
  </script>
  <script async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js" type="text/javascript">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-978727682&amp;l=dataLayer&amp;cx=c" type="text/javascript">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-6JRW0393LB&amp;cx=c&amp;_slc=1" type="text/javascript">
  </script>
  <script data-ads-env="prod" data-ads-portal-id="6951874" data-hsjs-env="prod" data-hsjs-hublet="na1" data-hsjs-portal="6951874" data-loader="hs-scriptloader" id="hs-ads-pixel-6951874" src="https://js.hsadspixel.net/fb.js" type="text/javascript">
  </script>
  <script crossorigin="anonymous" data-hsjs-env="prod" data-hsjs-hublet="na1" data-hsjs-portal="6951874" data-loader="hs-scr

In [7]:
job_opportunities = soup.select('.link-detalhes-vaga')
job_opportunities

[<a class="link-detalhes-vaga" data-id-vaga="2588010" href="/vagas/v2588010/analista-de-dados-junior" id="v2588010" onclick="ga('send', 'event', 'Pesquisa', 'Vaga_Visualizada');" title="Analista de Dados Júnior">
 <mark>Analista</mark> de <mark>Dados</mark> Júnior
 </a>,
 <a class="link-detalhes-vaga" data-id-vaga="2596076" href="/vagas/v2596076/analista-de-dados-senior-l" id="v2596076" onclick="ga('send', 'event', 'Pesquisa', 'Vaga_Visualizada');" title="Analista de Dados Senior L">
 <mark>Analista</mark> de <mark>Dados</mark> Senior L
 </a>,
 <a class="link-detalhes-vaga" data-id-vaga="2597008" href="/vagas/v2597008/analista-de-dados" id="v2597008" onclick="ga('send', 'event', 'Pesquisa', 'Vaga_Visualizada');" title="Analista de Dados">
 <mark>Analista</mark> de <mark>Dados</mark>
 </a>,
 <a class="link-detalhes-vaga" data-id-vaga="2586839" href="/vagas/v2586839/analista-de-dados-pleno" id="v2586839" onclick="ga('send', 'event', 'Pesquisa', 'Vaga_Visualizada');" title="Analista de Da

In [8]:
print(len(job_opportunities))

292


In [9]:
# def get_job_description(job_description):

#     job_info_description = {}

#     ul_count = len(job_description)
#     try:

#         job_info_description['descricao'] = []
#         job_info_description['requisitos'] = []

#         if ul_count >= 2:

#             description_list = job_description[0]

#             li_description = description_list.find_all('li')

#             for li in li_description:
#                 job_info_description['descricao'].append(li.text)

            
#             requirements_list = job_description[1:]
            
#             for requirements in requirements_list:
#                 li_requirements = requirements.find_all('li')

#                 for li in li_requirements:
#                     job_info_description['requisitos'].append(li.text)
#         else:
#             raise ValueError('Not enough lists to build the description field')
        
    
#     except:
#         return None

    
#     return job_info_description


def get_job_opportunity_info(job_url):

    r = requests.get('https://www.vagas.com.br' + job_url)

    job_page = BeautifulSoup(r.content, 'html')

    
    job_info = {}

    job_info['site_da_vaga'] = 'Vagas.com'
    job_info['link'] = 'https://www.vagas.com.br' + job_url

    try:
        job_info['data_publicação'] = job_page.select('.job-breadcrumb li')[0].text.strip()
    except:
        job_info['data_publicação'] = None

    
    job_info['data_coleta'] = datetime.today().strftime('%Y-%m-%d')


    try:
        job_info['titulo_da_vaga'] = job_page.select('.job-shortdescription__title')[0].text.strip()
    except:
        job_info['titulo_da_vaga'] = None


    try:
        job_info['local'] = job_page.select('.info-localizacao')[0].text.strip()
    except:
        job_info['local'] = None


    try:
        job_info['senioridade'] = job_page.select('.job-hierarchylist')[0].select_one('span').get('aria-label')
    except:
        job_info['senioridade'] = None


    try:
        job_info['modalidade'] = job_page.select('')[0].text.strip()
    except:
        job_info['modalidade'] = None

    try:
        job_info['contrato'] = job_page.select('')[0].text.strip()
    except:
        job_info['contrato'] = None


    try:
        job_info['beneficios'] = []
        benefits = job_page.select('.job-benefits__list')[0].find_all('span')

        for benefit in benefits:
            job_info['beneficios'].append(benefit.text)

    except:
        job_info['beneficios'] = None
    

    
    try:
        job_info['regime'] = job_page.select('.info-modelo-contratual')[0].text.strip()
    except:
        job_info['regime'] = None

    try:
        job_info['contrato'] = job_page.select('')[0].text.strip()
    except:
        job_info['contrato'] = None

    try:
        job_info['codigo_vaga'] = job_page.select('.job-breadcrumb li')[1].text.strip()
    except:
        job_info['contrato'] = None


    try:
        job_info['descricao'] = job_page.select('.job-tab-content.job-description__text.texto')[0].text.strip()
    except:
        job_info['descricao'] = None


    # job_description = job_page.select('.job-tab-content.job-description__text.texto')[0].find_all('ul')
    
    # description_info = get_job_description(job_description)

    # try:
    #     job_info.update(description_info)

    # except:
    #     job_info['descricao'] = job_page.select('.job-tab-content.job-description__text.texto')[0].text.strip()
    #     job_info['requisitos'] = None
        


    return job_info

In [10]:
#get_job_opportunity_info('/vagas/v2594750/analista-de-processos')
get_job_opportunity_info('/vagas/v2597008/analista-de-dados')

{'site_da_vaga': 'Vagas.com',
 'link': 'https://www.vagas.com.br/vagas/v2597008/analista-de-dados',
 'data_publicação': 'Publicada em 29/12/2023',
 'data_coleta': '2024-01-23',
 'titulo_da_vaga': 'Analista de Dados',
 'local': 'Minas Gerais',
 'senioridade': 'Pleno',
 'modalidade': None,
 'contrato': None,
 'beneficios': ['Assistência médica',
  'Assistência odontológica',
  'Consignado',
  'Horário flexível',
  'Participação nos Lucros ou Resultados',
  'Seguro de vida',
  'Vale-alimentação',
  'Vale-transporte'],
 'regime': 'Regime CLT',
 'codigo_vaga': 'v2597008',
 'descricao': 'Descrição\nDescrição: Atividades:\nRealizar através de técnicas consistentes o tratamento de grande volume de dados;\nAplicar técnicas de analise, modelagem e relacionamento de dados, gerando informações pertinentes aos negócios da empresa e a tomada de decisão;\nProduzir dashboards, relatórios gerenciais em ferramentas do Power BI;\nAcompanhar e apontar alertas baseado em alterações repentinas de performanc

In [11]:
job_list = []

for index, job in enumerate(job_opportunities):
    
    if index % 20 == 0:
        print(index)
        
    job_url = job['href']
    job_info = get_job_opportunity_info(job_url)

    job_list.append(job_info)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280


In [12]:
job_list[10]

{'site_da_vaga': 'Vagas.com',
 'link': 'https://www.vagas.com.br/vagas/v2596247/consultor-de-telecom-analista-de-banco-de-dados',
 'data_publicação': 'Publicada em 27/12/2023',
 'data_coleta': '2024-01-23',
 'titulo_da_vaga': 'Consultor de Telecom (Analista de Banco de Dados)',
 'local': 'Distrito Federal',
 'senioridade': 'Pleno',
 'modalidade': None,
 'contrato': None,
 'beneficios': ['Assistência médica',
  'Assistência odontológica',
  'Auxílio academia',
  'Auxílio creche',
  'Auxílio farmácia',
  'Ginástica laboral',
  'Participação nos Lucros ou Resultados',
  'Previdência privada',
  'Seguro de vida',
  'Vale-alimentação',
  'Vale-transporte'],
 'regime': 'Regime CLT',
 'codigo_vaga': 'v2596247',
 'descricao': 'Descrição\nDescrição: Suas principais atividades serão:\nProgramação com Python para realizar análises e automações em conjunto de dados;\nRealizar processamento de dados com Pandas e Apache Spark;\nRealização/construção de consultas SQL em bancos de dados e sites;\nAnal

In [13]:
job_list[8]

{'site_da_vaga': 'Vagas.com',
 'link': 'https://www.vagas.com.br/vagas/v2596262/consultor-de-telecom-senior-analista-de-dados',
 'data_publicação': 'Publicada em 27/12/2023',
 'data_coleta': '2024-01-23',
 'titulo_da_vaga': 'Consultor de Telecom Sênior (Analista de Dados)',
 'local': 'Distrito Federal',
 'senioridade': 'Sênior',
 'modalidade': None,
 'contrato': None,
 'beneficios': ['Assistência médica',
  'Assistência odontológica',
  'Auxílio academia',
  'Auxílio creche',
  'Auxílio farmácia',
  'Ginástica laboral',
  'Participação nos Lucros ou Resultados',
  'Previdência privada',
  'Seguro de vida',
  'Vale-alimentação',
  'Vale-transporte'],
 'regime': 'Regime CLT',
 'codigo_vaga': 'v2596262',
 'descricao': 'Descrição\nDescrição: Suas principais atividades serão:\nConstrução de dashboards e relatórios com PowerBI, Python em nível avançado;\nConstrução de processos ETL com ferramentas. Exemplo: Python, Spark e Pandas;\nConhecimento em PL/SQL. Exemplo: Banco Postgres, MySQL em ní

In [14]:
import json

with open('../data/data_raw/vagas.json', 'w', encoding='utf-8') as file:
    json.dump(job_list, file, ensure_ascii=False, indent=4)

In [15]:
with open('../data/data_raw/vagas.json', 'r', encoding='utf-8') as file:
    job_list = json.load(file)

In [16]:
job_list[0]

{'site_da_vaga': 'Vagas.com',
 'link': 'https://www.vagas.com.br/vagas/v2588010/analista-de-dados-junior',
 'data_publicação': 'Publicada em 29/11/2023',
 'data_coleta': '2024-01-23',
 'titulo_da_vaga': 'Analista de Dados Júnior',
 'local': 'Rio de Janeiro',
 'senioridade': 'Júnior/Trainee',
 'modalidade': None,
 'contrato': None,
 'beneficios': ['Assistência médica',
  'Assistência odontológica',
  'Auxílio academia',
  'Auxílio desenvolvimento',
  'Convênio com empresas parceiras',
  'Programa de remuneração variável',
  'Programa de treinamentos',
  'Refeitório',
  'Seguro de vida',
  'Vale-alimentação',
  'Vale-refeição',
  'Vale-transporte'],
 'regime': 'Regime CLT',
 'codigo_vaga': 'v2588010',
 'descricao': 'Descrição\nDescrição: Propósito do Cargo:Acompanhar indicadores e elaborar relatórios a partir de bancos de dados, análise de dados, e outras estratégias, mediante informações obtidos nos sistemas internos da empresa visando aumentar a eficiência da área e a qualidade dos ser

In [17]:
df_vagas = pd.DataFrame(job_list)
df_vagas

,site_da_vaga,link,data_publicação,data_coleta,titulo_da_vaga,local,senioridade,modalidade,contrato,beneficios,regime,codigo_vaga,descricao
0,Vagas.com,https://www.vagas.com.br/vagas/v2588010/analis...,Publicada em 29/11/2023,2024-01-23,Analista de Dados Júnior,Rio de Janeiro,Júnior/Trainee,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2588010,Descrição\nDescrição: Propósito do Cargo:Acomp...
1,Vagas.com,https://www.vagas.com.br/vagas/v2596076/analis...,Publicada em 26/12/2023,2024-01-23,Analista de Dados Senior L,Vitória,Pleno,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2596076,Descrição\nDescrição: A Adecco é uma multinac...
2,Vagas.com,https://www.vagas.com.br/vagas/v2597008/analis...,Publicada em 29/12/2023,2024-01-23,Analista de Dados,Minas Gerais,Pleno,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2597008,Descrição\nDescrição: Atividades:\nRealizar at...
3,Vagas.com,https://www.vagas.com.br/vagas/v2586839/analis...,Publicada em 27/11/2023,2024-01-23,Analista de Dados Pleno,São Paulo,Pleno,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2586839,Descrição\nDescrição: Responsabilidades e atri...
4,Vagas.com,https://www.vagas.com.br/vagas/v2599242/analis...,Publicada em 08/01/2024,2024-01-23,Analista de Dados de Survey,Macaé,Sênior,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2599242,Descrição\nDescrição: Resumo:O analista de dad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,Vagas.com,https://www.vagas.com.br/vagas/v2602905/tecnic...,Publicada há 6 dias,2024-01-23,Técnico Laboratório - Laboratório Hospital - H...,Pernambuco,Técnico,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2602905,Descrição\nDescrição: Requisitos:- Formação: T...
288,Vagas.com,https://www.vagas.com.br/vagas/v2603430/engenh...,Publicada há 5 dias,2024-01-23,Engenheiro de Software Jr. - TI - Arquitetura ...,São Paulo,Sênior,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2603430,Descrição\nDescrição: A área de Arquitetura e ...
289,Vagas.com,https://www.vagas.com.br/vagas/v2603434/engenh...,Publicada há 5 dias,2024-01-23,Engenheiro de Software Pl. - TI - Arquitetura ...,São Paulo,Pleno,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2603434,Descrição\nDescrição: A área de Arquitetura e ...
290,Vagas.com,https://www.vagas.com.br/vagas/v2603436/engenh...,Publicada há 5 dias,2024-01-23,Engenheiro de Software Sr. - TI - Arquitetura ...,São Paulo,Sênior,None,None,"[Assistência médica, Assistência odontológica,...",Regime CLT,v2603436,Descrição\nDescrição: Requisitos:Formação obri...


In [18]:
df_vagas['regime'].value_counts()

regime
Regime CLT         270
Temporário          14
Pessoa Jurídica      5
Estágio              1
Name: count, dtype: int64

In [19]:
df_vagas['senioridade'].value_counts()

senioridade
Pleno                   120
Júnior/Trainee           76
Sênior                   74
Auxiliar/Operacional     12
Técnico                   7
Estágio                   1
Name: count, dtype: int64

In [20]:
df_vagas.to_excel('data/vagas_vagas_com.xlsx', index=False)

In [79]:
#r = requests.get('https://www.vagas.com.br/vagas/v2594848/analista-de-dados-senior')
#r = requests.get('https://www.vagas.com.br/vagas/v2597008/analista-de-dados')
#r = requests.get('https://www.vagas.com.br/vagas/v2594750/analista-de-processos')
#r = requests.get('https://www.vagas.com.br/vagas/v2596076/analista-de-dados-senior-l')
#r = requests.get('https://www.vagas.com.br/vagas/v2586839/analista-de-dados-pleno')
r = requests.get('https://www.vagas.com.br/vagas/v2588010/analista-de-dados-junior')


page = BeautifulSoup(r.content, 'html')

In [80]:
job_description = page.select('.job-tab-content.job-description__text.texto')[0].find_all('ul')
job_description

[<ul>
 <li>Superior completo;</li>
 <li>Experiência em elaboração e análise de dashboard;</li>
 <li>Conhecimento de Power BI.</li>
 </ul>]

In [81]:
len(job_description)

1

In [84]:
emprego = {}

In [85]:
ul_count = len(job_description)
try:

    emprego['descricao'] = []
    emprego['requisitos'] = []

    if ul_count >= 2:

        description_list = job_description[0]

        li_description = description_list.find_all('li')

        for li in li_description:
            emprego['descricao'].append(li.text)

        
        requirements_list = job_description[1:]
        
        for requirements in requirements_list:
            li_requirements = requirements.find_all('li')

            for li in li_requirements:
                emprego['requisitos'].append(li.text)
    else:
        raise ValueError('Not enough lists to build the description field')
except:
    emprego['descricao'] = None
    emprego['requisitos'] = None

In [86]:
emprego

{'descricao': None, 'requisitos': None}